In [1]:
import numpy as np

In [2]:
# resampling

elements = ['one', 'two', 'three']
weights = [0.2, 0.3, 0.5]

from numpy.random import choice
print(choice(elements))
print(choice(elements, size = 10, replace = True, p = weights))
# element weight

one
['three' 'two' 'one' 'three' 'two' 'three' 'three' 'three' 'one' 'three']


In [3]:
x = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size = 0.3, random_state = 11)

In [6]:
x_train[:2]

array([[0.29850933, 0.48312843, 0.11111111, 0.        , 0.        ,
        0.        , 0.5       , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.19829638, 0.16883676, 0.        , 0.        , 1.        ,
        0.        , 0.25      , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [7]:
y_train[:10]

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [9]:
eclf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 2), 
 n_estimators = 500, learning_rate = 0.1)

In [10]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, x_train, y_train, cv = 5).mean()

0.7862265932336744

In [11]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
AdaBoostClassifier()

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [13]:
params = {
    "base_estimator__criterion" : ["gini", "entropy"],
    "base_estimator__max_features" : [7,8,],
    "base_estimator__max_depth" : [1,2,3,4,5],
    "n_estimators": [23,24, 25, 26, 27],
    "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
    }

# "base_estimator__criterion" : ["gini", "entropy"], =>
# "base_estimator__max_features" : [7,8,],
# "base_estimator__max_depth" : [1,2], => 3 .
# ==> base estimator(ensemble model) parameter (decision tree parameter)
# "n_estimators": [23,24, 25, 26, 27],
# "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]

In [20]:
# gridsearch(=>GridSearchCV), cross_validation(=>cv), ensemble(=>eclf)
# gridsearchcv operation

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator = eclf, param_grid = params, cv = 5, n_jobs=-1)
grid.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.1, n_estimators=500, random_state=None),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'base_estimator__criterion': ['gini', 'entropy'], 'base_estimator__max_features': [7, 8], 'base_estimator__max_depth': [1, 2, 3, 4, 5], 'n_estimators': [23, 24, 25, 26, 27], 'learning_rate': [0.4, 0.45, 0.5, 0.55, 0.6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

* gridsearch , cpu => n_jobs
 - 사용할 cpu의 갯수 n_jobs로 표현
 - n_jobs = -1 => 사용할 수 있는 최대한의 cpu 사용

In [21]:
# best soore
grid.best_score_

0.8263665594855305

In [23]:
# best parameters
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.4,
 'n_estimators': 24}

In [ ]:
grid.best_estimator_.feature_importances_

In [ ]:
grid.score(x_train, y_train)

In [ ]:
grid.score(x_test, y_test)

In [25]:
pred = grid.predict(x_test)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

In [26]:
# confusion matrix
confusion_matrix(y_test, pred)

array([[144,  17],
       [ 26,  80]], dtype=int64)

In [27]:
# classification report
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.85      0.89      0.87       161
         1.0       0.82      0.75      0.79       106

   micro avg       0.84      0.84      0.84       267
   macro avg       0.84      0.82      0.83       267
weighted avg       0.84      0.84      0.84       267

